# Summary

To see what variables in the dataframe mean:
https://github.com/maksimhorowitz/nflscrapR/blob/master/R/scrape_play_by_play.R

To see how to pull all the data at a player level

# Data Schema 

## Grain 

Data will be at a (game,player) grain 


In [2]:
import os
# Paths
path_dict = {}
path_dict['root'] = '/Users/ryanstevens/Documents/github/nflscrapR-data/'
path_dict['games_data'] = os.path.join(path_dict['root'],'games_data')
path_dict['pbp_data'] = os.path.join(path_dict['root'],'play_by_play_data')

In [3]:
import pandas as pd

# Get regular season play by play data
year = 2009
path_to_pbp_data = os.path.join(path_dict['pbp_data'],'regular_season','reg_pbp_{year}.csv'.format(year=year))
reg_season = pd.read_csv(path_to_pbp_data)

reg_season.columns

Index(['play_id', 'game_id', 'home_team', 'away_team', 'posteam',
       'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'game_date',
       ...
       'penalty_player_id', 'penalty_player_name', 'penalty_yards',
       'replay_or_challenge', 'replay_or_challenge_result', 'penalty_type',
       'defensive_two_point_attempt', 'defensive_two_point_conv',
       'defensive_extra_point_attempt', 'defensive_extra_point_conv'],
      dtype='object', length=256)